In [32]:
'''

This file is designed to download many webpages from commoncrawl in parallel

integrated version: makes it simpler to apply to any list

'''


from comcrawl import IndexClient
import json
import time

In [33]:
from joblib import Parallel, delayed
import os

client = IndexClient()
all_indexes = client.indexes

In [37]:




def get_result(url, year):
    target_indexes = [index for index in all_indexes if int(index.split('-')[0]) <= int(year)]
    client = IndexClient(indexes = target_indexes)
    
    client.search(url, threads=32)

    # try downloading a few times in case we run into issues downloading
    for _ in range(5):
        client.download(threads=32)
        
        if len(client.results) > 0:
            break
    
    
    return client.results

def job_code(inp):
    slice_i, url_list, path, retry_empty = inp
    
    out_file = path + 'slice_{}_urls_out.json'.format(slice_i)
    backup_file = path + 'slice_{}_urls_out_backup.json'.format(slice_i)
    
    if not os.path.isfile(out_file):
        with open(out_file,'w') as f:
            f.write(json.dumps({}))
    
    try:
        with open(out_file) as f:
            out_dict = json.load(f)
    except:
        out_dict= {}
    try:
        with open(backup_file) as f:
            out_dict_ = json.load(f)
    except:
        out_dict_ = {}
        
    if len(out_dict) < len(out_dict_):
        out_dict = out_dict_
    
    for (url, year) in url_list:
        
        if url not in out_dict.keys() or (retry_empty and len(out_dict[url]) == 0):
            try:
                out_dict[url] = get_result(url, year)

            except:
                pass
            with open(backup_file, 'w') as f:
                json.dump(out_dict, f)        
            with open(out_file, 'w') as f:
                json.dump(out_dict, f)
    with open(backup_file, 'w') as f:
        json.dump(out_dict, f)        
    with open(out_file, 'w') as f:
        json.dump(out_dict, f)

def robust_job_code(inp):
    slice_i, url_list, path, retry_empty = inp
    
    out_file = path + 'slice_{}_urls_out.json'.format(slice_i)
    backup_file = path + 'slice_{}_urls_out_backup.json'.format(slice_i)
    
    if not os.path.isfile(out_file):
        with open(out_file,'w') as f:
            f.write(json.dumps({}))
    
    try:
        with open(out_file) as f:
            out_dict = json.load(f)
    except:
        try:
            with open(backup_file) as f:
                out_dict = json.load(f)
        except:
            out_dict = {}
        
    # try running job code 5 times to catch
    # any stragglers
    for i in range(3):
        #try:
        if True:
            print('trying {}'.format(slice_i))
            job_code(inp)
        #except:
        if False:
            print('fail {}'.format(slice_i))
        # sleep for 5 seconds in case we need to kill it
        time.sleep(5)
    

In [38]:
'''

Slice up input urls for parallel loading

splitting this up between tricycle and bicycle


'''

urls_in = 'urls_to_load_processing_3.json'
path_out = 'processing_3/'


if (not os.path.isdir(path_out)):
    os.mkdir(path_out)
    

n_slices =64

with open(urls_in) as f:
    urls_to_load = json.load(f)


total_keys = list(urls_to_load.keys())
step = int(len(urls_to_load)/n_slices + 1)
#step = 5
slices = []

for j in range(n_slices):
    keys = total_keys[j*step:(j+1)*step]

    slices += [[(key, urls_to_load[key]) for key in keys]]

In [ ]:
slice_range =  range(0,n_slices)


# tuple input it slice_i, slice, path_out, retry_empty
Parallel(n_jobs=32)(delayed(robust_job_code)((i, slices[i], path_out, True)) for i in slice_range)

In [202]:
''' Down here is surgery for broken jsons'''

i = 20
with open('slice_{}_urls_out.json'.format(i)) as f:
    line = f.read()

In [203]:
ind = 0
for j in range(len(line), 0, -1):
    if line[j:j+10].startswith('": ['):
        ind = j
        break
print(len(line))
print(ind)
print(j)

11647499
5252468
5252468


In [23]:
result = get_result('https://www.washingtonpost.com/news/volokh-conspiracy/wp/2014/08/01/does-israel-have-to-give-free-power-to-gaza/',2015
          )

In [19]:
import requests

In [20]:
r = requests.get('https://www.washingtonpost.com/news/volokh-conspiracy/wp/2014/08/01/does-israel-have-to-give-free-power-to-gaza/', auth=('user', 'pass'))

In [25]:
t1 = r.text
t2 = result[0]['html']

In [28]:
with open('t1.txt','w') as f:
    f.write(t1)
    
with open('t2.txt','w') as f:
    f.write(t2)

In [30]:
urls_to_load

{'https://www.forbes.com/2008/01/28/music-madonna-hollywood-biz-cz_dp_0129musicwomen.html#73b1e2cb67b3': '2019',
 'http://www.digitalspy.co.uk/soaps/sitenews/a130011/soaplands-joiners-movers-and-leavers.html#~p2usQoUQliXijX': '2015',
 'http://latimesblogs.latimes.com/babylonbeyond/2009/04/egypt-hizbollah-dismissed-as-terrorist.html#sthash.okhZkwqI.dpuf': '2017',
 'http://observer.com/2012/08/developing-three-arrested-at-pussy-riot-demonstration/#axzz32AUfrZh4': '2014',
 'https://en.wikipedia.org/wiki/Bangor,_County_Down#The_Troubles': '2015',
 'http://www.billboard.com/#/album/lucero/cerca-de-ti/288554<ref>': '2010',
 'http://www.telegraph.co.uk/topics/weather/8170360/UK-snow-roads-and-schools-closed-as-more-Arctic-weather-forecast.html': '2010',
 'https://books.google.com/books?id=5LI5AQAAMAAJ&dq=andranik%20gendarme&pg=RA2-PA90#v=onepage&q=andranik%20gendarme&f=false': '2017',
 'https://www.theguardian.com/australia-news/2015/jun/12/french-ambassador-lauds-shared-values-with-australia